In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt

In [2]:
env = gym.make('MountainCar-v0')
numberOfActions = env.action_space.n

D:\SOFTWARE\Anaconda3\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
D:\SOFTWARE\Anaconda3\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
actionSpace = [0, 1, 2]
numberOfStates = 20 ** 2
GAMMA = 0.9
ALPHA = 0.05

In [4]:
def createBins():

    bins = np.zeros((2, 20))
    bins[0] = np.linspace(-1.20, 0.6, 20)
    bins[1] = np.linspace(-0.07, 0.07, 20)
    return bins

In [5]:
def assignBins(observation, bins):
    state = np.zeros(2)
    for i in range(2):
        state[i] = np.digitize(observation[i], bins[i])
    return state

In [6]:
def max_dict(d):
    max_v = float('-inf')
    for key, val in d.items():
        if val > max_v:
            max_v = val
            max_key = key
    return max_key, max_v

In [7]:
#initilizing the action value function
def initQ():
  Q = {}

  for i in range(20):
    for j in range(20):
      Q[(i,j)] = {}
      for action in range(env.action_space.n):
        Q[(i,j)][action] = float(0)
  return Q

In [8]:
#initilize a random deterministic target policy
def initTargetPolicy():
  pi = {}

  for i in range(20):
    for j in range(20):
      pi[(i,j)]= np.random.choice(actionSpace)
  return pi

In [9]:
def initReturns():
    returns = {}
    
    for i in range(20):
        for j in range(20):
            returns[(i,j)] = {}
            for action in range(env.action_space.n):
                returns[(i,j)][action] = []
    return returns

In [20]:
def Expected_SARSA(env,Q, bins,eps):
    obs = env.reset()
    state = tuple(assignBins(obs, bins))
    # prev_screen = env.render(mode='rgb_array')
    # plt.imshow(prev_screen)
    count = 0
    complete = 0
    while True:
        count += 1
        if np.random.uniform() < eps:
            action = env.action_space.sample()  # epsilon greedy
        else:
            action = max_dict(Q[state])[0]

        obs, reward, done, info = env.step(action)


        if done == True:
            if count < 200:
                reward = 1000
                complete = 1

        expected_q = 0
        greedy_actions = 0
        new_state = tuple(assignBins(obs, bins))
        max_action, max_q = max_dict(Q[new_state])
        
        for i in actionSpace:
            if Q[new_state][i] == max_q:
                greedy_actions += 1

        non_greedy_action_probability = eps / len(actionSpace)
        greedy_action_probability = ((1 - eps) / greedy_actions) + non_greedy_action_probability

        for i in actionSpace:
            if Q[new_state][i] == max_q:
                expected_q += Q[new_state][i] * greedy_action_probability
            else:
                expected_q += Q[new_state][i] * non_greedy_action_probability


        Q[state][action] += ALPHA * (reward + GAMMA * expected_q - Q[state][action])
        state = new_state
        if done == True:
            break;
    return complete

In [21]:
def evaluate_and_plot_parameters(cumulative_completion, training_episodes):
    title = f'total episodes vs completed episodes'
    print(f'Evaluated {title}')
    line, = plt.plot(
        np.arange(1, training_episodes + 1),
        cumulative_completion,
        label=title)
    return line

In [23]:
bins = createBins()
Q = initQ()

In [24]:
env = gym.make('MountainCar-v0')
SARSA_completeList = []
completed = 0
eps = 0.05
mem = 0
training_episodes = 5000

In [25]:
for i in range(training_episodes):
    complete = Expected_SARSA(env, Q, bins, eps)
    if i % 100 == 0 and i != 0:
        print("i = " + str(i) + ", and completed over past 100 episode " + str(completed - mem))
        mem = completed

    completed += complete
    SARSA_completeList.append(completed)

evaluate_and_plot_parameters(SARSA_completeList, training_episodes)
plt.show()


i = 100, and completed over past 100 episode 0
i = 200, and completed over past 100 episode 0
i = 300, and completed over past 100 episode 0
i = 400, and completed over past 100 episode 0
i = 500, and completed over past 100 episode 0
i = 600, and completed over past 100 episode 0
i = 700, and completed over past 100 episode 0
i = 800, and completed over past 100 episode 0
i = 900, and completed over past 100 episode 1
i = 1000, and completed over past 100 episode 0
i = 1100, and completed over past 100 episode 2
i = 1200, and completed over past 100 episode 2
i = 1300, and completed over past 100 episode 2
i = 1400, and completed over past 100 episode 1
i = 1500, and completed over past 100 episode 46
i = 1600, and completed over past 100 episode 25
i = 1700, and completed over past 100 episode 21
i = 1800, and completed over past 100 episode 6
i = 1900, and completed over past 100 episode 32
i = 2000, and completed over past 100 episode 74
i = 2100, and completed over past 100 episod

NameError: name 'onPolicy_SARSA_completeList' is not defined

In [29]:
eps = 0
avg = 0
for i in range(100):
    complete = Expected_SARSA(env, Q, bins, eps)
    avg += complete
print(avg / 100)

0.92
